In [1]:
import requests
import pandas as pd
import numpy as np
from sodapy import Socrata

In [2]:
# Use API to get occupancy data from website
client = Socrata("data.seattle.gov",None,timeout=1000000000)

# Order by occupancydatetime, sourcelementkey, pull first 200,000 entries 
results = client.get('hiyf-7edq',order='occupancydatetime DESC, sourceelementkey',limit=200000)
results_df2 = pd.DataFrame.from_records(results)

In [3]:
# Save to new dataframe so I can manipulate data without needing to pull again
results_df = results_df2
results_df

,occupancydatetime,paidoccupancy,blockfacename,sideofstreet,sourceelementkey,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,location
0,2023-02-25T21:59:00.000,0,1ST AVE N BETWEEN THOMAS ST AND HARRISON ST,W,1037,240,12,Uptown,Edge,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3555141..."
1,2023-02-25T21:59:00.000,0,1ST AVE N BETWEEN HARRISON ST AND REPUBLICAN ST,W,1041,240,7,Uptown,Edge,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3555045..."
2,2023-02-25T21:59:00.000,0,1ST AVE N BETWEEN REPUBLICAN ST AND MERCER ST,W,1045,240,9,Uptown,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3554985..."
3,2023-02-25T21:59:00.000,0,1ST AVE N BETWEEN REPUBLICAN ST AND MERCER ST,E,1046,120,12,Uptown,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3553161..."
4,2023-02-25T21:59:00.000,3,10TH AVE BETWEEN E MADISON ST AND E SENECA ST,W,1229,120,5,Pike-Pine,NaN,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3194595..."
...,...,...,...,...,...,...,...,...,...,...,...
199995,2023-02-25T17:23:00.000,5,PINE ST BETWEEN MINOR AVE AND MELROSE AVE,SE,80222,120,3,Pike-Pine,NaN,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3282467..."
199996,2023-02-25T17:23:00.000,1,QUEEN ANNE AVE N BETWEEN JOHN ST AND THOMAS ST,W,80273,240,9,Uptown,Edge,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3568237..."
199997,2023-02-25T17:23:00.000,4,QUEEN ANNE AVE N BETWEEN JOHN ST AND THOMAS ST,E,80274,240,11,Uptown,Edge,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3566615..."
199998,2023-02-25T17:23:00.000,2,QUEEN ANNE AVE N BETWEEN THOMAS ST AND HARRISO...,W,80277,240,8,Uptown,Edge,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3568151..."


In [4]:
# Convert column to datetime format
results_df['occupancydatetime'] = pd.to_datetime(results_df['occupancydatetime'])

# Sort dataframe by date (most recent first)
results_df = results_df.sort_values('occupancydatetime',ascending=False)

# Get unique values for each location (pick first row based on sourcelementkey)
results_df = results_df.drop_duplicates(subset=['sourceelementkey'])

In [5]:
# Create new column for coordinates
results_df['coordinates'] = results_df['location'].apply(lambda x: x.get('coordinates'))

In [6]:
# Create new columns for latitude and longitutde
results_df['latitude'], results_df['longitude'] = results_df.coordinates.str

/tmp/ipykernel_7474/2101011984.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  results_df['latitude'], results_df['longitude'] = results_df.coordinates.str


In [7]:
# Convert column to int64 format
results_df['sourceelementkey'] = results_df['sourceelementkey'].astype(np.int64)

In [8]:
# Import blockface data (downloaded from https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::blockface/about)
geo_parking = pd.read_csv('Blockface.csv')

In [9]:
# Merge using element keys (left join)
merged_parking = pd.merge(left=geo_parking, right=results_df,how='left',left_on='ELMNTKEY', right_on='sourceelementkey')

In [10]:
merged_parking

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,sourceelementkey,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,location,coordinates,latitude,longitude
0,1,40194,17631,13640,290,NE 80TH ST BETWEEN 29TH AVE NE AND DEAD END 2,S,NaN,2900,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2766,2741,930,380,19TH AVE SW BETWEEN SW CHARLESTOWN E ST AND SW...,E,NaN,3800,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,37209,14676,11485,160,E PROSPECT ST BETWEEN 16TH E AVE E AND 17TH AVE E,N,NaN,1600,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,85425,17479,13560,350,NE 73RD ST BETWEEN DEAD END AND 38TH W AVE NE,N,NaN,3500,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,37217,14681,11485,210,E PROSPECT ST BETWEEN 21ST AVE E AND 22ND AVE E,N,NaN,2100,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47850,72018,121364,164794,16695,352,S ORCAS ST BETWEEN 35TH E AVE S AND M L KING J...,S,NaN,3500,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47851,72020,84398,16493,12975,233,NE 125TH ST BETWEEN 24TH W AVE NE AND 24TH E A...,S,NaN,2400,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47852,72022,21577,21311,16970,4,S SPOKANE NR ST BETWEEN EAST MARGINAL ER WAY S...,N,NaN,0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47853,72024,49789,4501,2095,1200,3RD AVE NW BETWEEN NW 120TH ST AND NW 122ND ST,W,NaN,12000,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# CHECK merge using element keys (inner join) - 1380 rows
# According to Blockface.csv, there are 1450 parking paid spots
# I think it is okay to have 1380 rows matching, since even when I increase the limits of occupancy data still only 1380 rows matching
# Tradeoff between speed and matching
merged_parking_inner = pd.merge(left=geo_parking, right=results_df,how='inner',left_on='ELMNTKEY', right_on='sourceelementkey')

In [12]:
merged_parking_inner

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,sourceelementkey,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,location,coordinates,latitude,longitude
0,34,8405,8293,4790,80,8TH AVE N BETWEEN VALLEY ST AND ALOHA ST,W,N08-08,800,Y,...,8405,600,7,South Lake Union,North,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3411456...","[-122.34114567, 47.6266943]",-122.341146,47.626694
1,164,81133,13145,10055,125,THOMAS ST BETWEEN PONTIUS AVE N AND YALE AVE N,N,N3-12,1200,Y,...,81133,120,6,South Lake Union,South,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3310457...","[-122.33104579, 47.62088727]",-122.331046,47.620887
2,243,76197,8288,4790,20,8TH AVE N BETWEEN JOHN ST AND THOMAS ST,W,N08-02,200,Y,...,76197,120,25,South Lake Union,South,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3411826...","[-122.34118265, 47.62032565]",-122.341183,47.620326
3,271,8622,8527,4890,220,9TH AVE BETWEEN WESTLAKE AVE AND DENNY WAY,NE,09-22,2200,Y,...,8622,240,3,Denny Triangle,North,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3387934...","[-122.33879349, 47.61827352]",-122.338793,47.618274
4,372,76202,8290,4790,40,8TH AVE N BETWEEN HARRISON ST AND REPUBLICAN ST,E,N08-04,400,Y,...,76202,120,3,South Lake Union,South,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3409512...","[-122.3409512, 47.62266064]",-122.340951,47.622661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,71769,24829,1850,445,140,14TH AVE BETWEEN E UNION ST AND E MADISON ST,W,E14-14,1400,Y,...,24829,120,9,Pike-Pine,NaN,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3143163...","[-122.31431635, 47.61341953]",-122.314316,47.613420
1376,71816,2054,1997,545,152,15TH AVE BETWEEN E MADISON ST AND E PINE ST,E,E15-15,1500,Y,...,2054,120,7,Pike-Pine,NaN,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3126995...","[-122.31269957, 47.61494753]",-122.312700,47.614948
1377,71894,34502,11886,8675,20,MINOR AVE N BETWEEN JOHN ST AND THOMAS ST,E,NMR-02,200,Y,...,34502,600,10,South Lake Union,South,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3328777...","[-122.33287778, 47.62025858]",-122.332878,47.620259
1378,71922,34509,11888,8675,40,MINOR AVE N BETWEEN HARRISON ST AND REPUBLICAN ST,W,NMR-04,400,Y,...,34509,600,14,South Lake Union,South,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3330980...","[-122.33309802, 47.62256499]",-122.333098,47.622565
